I start just recovering the results of the previous notebook

In [1]:
import pandas as pd
result = pd.read_csv("Coordinates.csv").drop("Unnamed: 0", axis = 1)

Let's download all the data about the venues in the neighborhoods. The result of this part will be written in a csv and uploaded prior to masking all the data and being uploaded. Skip next cell if you are looking at the notebook on github

In [5]:
CLIENT_ID = 'MY_ID'
CLIENT_SECRET = 'MY_SECRET'
VERSION = '20180605' # Foursquare API version

import requests
from pandas.io.json import json_normalize

radius = 500
LIMIT = 1000
neighborhood_longitude = result.iloc[0]["Longitude"]
neighborhood_latitude = result.iloc[0]["Latitude"]
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)

def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    
    venues_list=[]
    total = len(names)
    counter = 1
    for name, lat, lng in zip(names, latitudes, longitudes):
        print("{}: {}/{:<100}".format(name, counter, total), end = "\r")
        counter += 1
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    print("{:<100}".format("Done!"))
    return(nearby_venues)

toronto_venues = getNearbyVenues(names = result['Neighborhood'],
                                    latitudes = result['Latitude'],
                                   longitudes = result['Longitude'])

toronto_venues.to_csv("Venues.csv")

Done!                                                                                               103                                                                                                                                         


Come here if you wanna try the notebook from github

In [10]:
toronto_venues = pd.read_csv("Venues.csv").drop(["Unnamed: 0"], axis = 1)

#There's probably some wrong data here. I'm really not sure but I clean them up just for percaution
toronto_venues = toronto_venues[toronto_venues["Venue Category"] != "Neighborhood"]

Convert the venue list in a table having for each neighborhood the number of venues of specific type

In [11]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot["Neighborhood"] = toronto_venues["Neighborhood"]
venues_count = toronto_onehot.groupby("Neighborhood").sum()
venues_count["Total"] = venues_count.sum(axis = 1)

total_table = result.join(venues_count, on = "Neighborhood").fillna(0)
total_table.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Accessories Store,Adult Boutique,Airport,American Restaurant,Antique Shop,...,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Total
0,M3A,North York,Parkwoods,43.751881,-79.330360,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
1,M4A,North York,Victoria Village,43.730419,-79.312820,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655140,-79.362648,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,22.0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723209,-79.451408,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,52.0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.664490,-79.393021,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0


Do the K-neighbour clustering using the venues number per neighborhood data and map the results!

In [14]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import numpy as np

n_clusters = 5

kmc = KMeans(n_clusters = n_clusters, random_state = 0).fit(total_table.iloc[:, 5:])

try:
    total_table.insert(5, 'Cluster Label', kmc.labels_)
except:
    total_table["Cluster Label"] = kmc.labels_

total_table

latitude = 43.6532
longitude = -79.3832
cluster_map = folium.Map(location=[latitude, longitude], zoom_start=10)


x = np.arange(n_clusters)
ys = [i + x + (i*x)**2 for i in range(n_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(total_table['Latitude'], total_table['Longitude'], total_table['Neighborhood'], total_table['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cluster_map)

cluster_map